In [9]:
import pandas as pd
import numpy as np
import re

In [10]:
# Load datasets
file_cpasf = "Cleaned_CPASF.csv"
file_cpmnt = "Cleaned_CPMNT.csv"
file_scheme_details = "SCHEME DETAILS.xlsx"

df_cpasf = pd.read_csv(file_cpasf)
df_cpmnt = pd.read_csv(file_cpmnt)

In [11]:
# Convert 'Date' column to datetime format
df_cpasf['Date'] = pd.to_datetime(df_cpasf['Date'])
df_cpmnt['Date'] = pd.to_datetime(df_cpmnt['Date'])

In [12]:
# Load SCHEME DETAILS.xlsx and process all sheets
xls = pd.ExcelFile(file_scheme_details)
sheet_names = xls.sheet_names
sheets_data = {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_names}

In [13]:
def clean_scheme_data(df):
    df = df.dropna(how='all')  # Remove completely empty rows
    df.columns = df.iloc[0]  # Set the first row as header
    df = df[1:].reset_index(drop=True)  # Remove the first row and reset index
    
    # Convert column names to strings before stripping whitespace
    df = df.rename(columns=lambda x: str(x).strip() if pd.notna(x) else x)
    
    # Extract numerical discount values using regex
    df['Discount'] = df.iloc[:, 4].astype(str).apply(lambda x: re.findall(r'\d+\.?\d*%', x))
    df['Discount'] = df['Discount'].apply(lambda x: x[0] if x else None)
    
    # Convert scheme period to start and end date
    df['Scheme Start'] = df.iloc[:, 2]
    df['Scheme End'] = df.iloc[:, 3]
    df['Scheme Start'] = pd.to_datetime(df['Scheme Start'], errors='coerce')
    df['Scheme End'] = pd.to_datetime(df['Scheme End'], errors='coerce')
    
    return df[['DATE', 'PRODUCT/PACK', 'Scheme Start', 'Scheme End', 'Discount', 'LOCATION/CLUSTER']]

In [14]:
# Process all sheets
processed_sheets = {sheet: clean_scheme_data(sheets_data[sheet]) for sheet in sheet_names}

C:\Users\sunil\AppData\Local\Temp\ipykernel_21676\1062734766.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Scheme Start'] = pd.to_datetime(df['Scheme Start'], errors='coerce')
C:\Users\sunil\AppData\Local\Temp\ipykernel_21676\1062734766.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Scheme End'] = pd.to_datetime(df['Scheme End'], errors='coerce')
C:\Users\sunil\AppData\Local\Temp\ipykernel_21676\1062734766.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Scheme Start'] = pd.to_datetime(df['Scheme Start'], errors='coerce')
C:\Users\sunil\AppData\

In [15]:
# Combine all structured data into a single DataFrame
scheme_data_final = pd.concat(processed_sheets.values(), ignore_index=True)

In [16]:
# Save cleaned scheme details
scheme_data_final.to_csv("Cleaned_Scheme_Details.csv", index=False)

print("Processed and cleaned scheme details saved to Cleaned_Scheme_Details.csv")

Processed and cleaned scheme details saved to Cleaned_Scheme_Details.csv


In [17]:
# Load the cleaned scheme details file
df_scheme = pd.read_csv("Cleaned_Scheme_Details.csv")

def clean_fully_scheme_details(df):
    # Fill missing values using forward-fill where applicable
    df['DATE'] = df['DATE'].fillna(method='ffill')
    df['PRODUCT/PACK'] = df['PRODUCT/PACK'].fillna(method='ffill')
    df['LOCATION/CLUSTER'] = df['LOCATION/CLUSTER'].fillna("UNKNOWN")
    
    # Fill missing discount values with 'No Discount'
    df['Discount'] = df['Discount'].fillna("No Discount")
    
    # Convert scheme period to datetime format
    df['Scheme Start'] = pd.to_datetime(df['Scheme Start'], errors='coerce')
    df['Scheme End'] = pd.to_datetime(df['Scheme End'], errors='coerce')
    
    # Fill missing scheme start and end dates logically
    df['Scheme Start'] = df['Scheme Start'].fillna(method='ffill')
    df['Scheme End'] = df['Scheme End'].fillna(method='ffill')
    
    return df

In [18]:
# Apply cleaning function
df_scheme = clean_fully_scheme_details(df_scheme)

C:\Users\sunil\AppData\Local\Temp\ipykernel_21676\1681325607.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['DATE'] = df['DATE'].fillna(method='ffill')
C:\Users\sunil\AppData\Local\Temp\ipykernel_21676\1681325607.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PRODUCT/PACK'] = df['PRODUCT/PACK'].fillna(method='ffill')
C:\Users\sunil\AppData\Local\Temp\ipykernel_21676\1681325607.py:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Scheme Start'] = df['Scheme Start'].fillna(method='ffill')
C:\Users\sunil\AppData\Local\Temp\ipykernel_21676\1681325607.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Scheme End'] 

In [19]:
# Save fully cleaned data
df_scheme.to_csv("Fully_Cleaned_Scheme_Details.csv", index=False)

In [20]:
# Drop 'DATE' column
df_scheme.drop(columns=['DATE'], inplace=True)

In [21]:
# Encode products (Prod A to Prod E)
for prod in ['Prod A', 'Prod B', 'Prod C', 'Prod D', 'Prod E']:
    df_scheme[prod] = df_scheme['PRODUCT/PACK'].apply(lambda x: 1 if prod in x else 0)

In [22]:
# City Mapping for encoding
city_mappings = {
    "BGLR": "Bangalore", "CAL": "Calcutta", "MUM": "Mumbai", "CHN": "Chennai",
    "DEL": "Delhi", "HYD": "Hyderabad", "KOL": "Kolkata"
}

def process_location(location):
    if pd.isna(location) or location == "UNKNOWN":
        return None
    location = location.upper()
    for short, full in city_mappings.items():
        location = location.replace(short, full)
    if "ALL INDIA" in location:
        return "All Cities"
    return location.split()[0]  # Extract first city name only

df_scheme['City'] = df_scheme['LOCATION/CLUSTER'].apply(process_location)
df_scheme = df_scheme.dropna(subset=['City'])

In [251]:
# One-hot encoding for cities
df_cities = pd.get_dummies(df_scheme['City'])
df_scheme = pd.concat([df_scheme, df_cities], axis=1)

df_scheme.to_csv("Final_Cleaned_Scheme_Details.csv", index=False)